# App Download Prediction Based on Customer Click Data
Kaggle competition data: https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection

In [2]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col
import os.path
from pyspark.sql.window import Window

from pyspark.ml.feature import SQLTransformer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

The dataset is loaded.

In [4]:
tr_raw = spark.read.parquet("/mnt/dbdata/fraud/train_parquet").orderBy(['ip', 'click_time'], ascending = True).persist()

In [5]:
display(tr_raw)

ip,app,device,os,channel,click_time,attributed_time,is_attributed
1,2,1,15,477,2017-11-09T01:25:31.000+0000,null,0
1,12,1,15,265,2017-11-09T01:25:35.000+0000,null,0
1,6,1,15,459,2017-11-09T01:25:36.000+0000,null,0
1,64,1,15,459,2017-11-09T01:25:36.000+0000,null,0
1,2,1,2,477,2017-11-09T01:47:39.000+0000,null,0
1,13,1,28,469,2017-11-09T03:37:16.000+0000,null,0
1,2,1,6,477,2017-11-09T03:59:36.000+0000,null,0
1,2,1,49,477,2017-11-09T04:05:42.000+0000,null,0
1,2,1,2,477,2017-11-09T04:08:39.000+0000,null,0
1,2,1,2,477,2017-11-09T04:10:32.000+0000,null,0


The number of rows of the data

In [7]:
tr_raw.count()

Out[ 4 ]: 184903890

The number of unique IP's

In [9]:
tr_raw.select('ip').distinct().count()

Out[ 5 ]: 277396

Click_time statistics

In [11]:
tr_raw.select(F.min('click_time')).show()

+-------------------+
 min(click_time)|
+-------------------+
2017-11-06 22:32:21|
+-------------------+

In [12]:
tr_raw.select(F.max('click_time')).show()

+-------------------+
 max(click_time)|
+-------------------+
2017-11-10 00:00:00|
+-------------------+

During night time, clickes are low. Periodic patterns in the clicks.

In [14]:
tr_raw.registerTempTable("tr_raw")

In [15]:
%sql
SELECT date_format(click_time, "MMM-dd HH:mm") as time FROM tr_raw

time
Nov-09 01:25
Nov-09 01:25
Nov-09 01:25
Nov-09 01:25
Nov-09 01:47
Nov-09 03:37
Nov-09 03:59
Nov-09 04:05
Nov-09 04:08
Nov-09 04:10


In [16]:
# display(tr_raw.select(F.unix_timestamp('click_time').alias('Time')))
%sql 
SELECT date_format(click_time, "MMM-dd HH:mm") as time FROM tr_raw

File "<command-4082240420257565>" , line 3 
 SELECT date_format(click_time, "MMM-dd HH:mm") as time FROM tr_raw 
 ^ 
 SyntaxError : invalid syntax

Conversions also follow the similar periodic patterns.

In [18]:
tr_raw.registerTempTable("tr_raw")

In [19]:
%sql 
SELECT unix_timestamp(attributed_time) as Converted_Time
FROM tr_raw
WHERE attributed_time IS NOT NULL

Converted_Time
1510220622
1510220727
1510260389
1510260410
1510261465
1510268177
1510268194
1510269354
1510269491
1510134871


In [20]:
%sql
SELECT unix_timestamp(click_time) as Click_Time
FROM tr_raw
WHERE ip = 7340

Click_Time
1510013253
1510013253
1510014120
1510014479
1510016512
1510018373
1510018380
1510020884
1510020885
1510020888


Aim: To Predict Conversion Rate (App Download Rate) per IP.

In [22]:
distinct_ip = tr_raw.select('ip').distinct()
tot_num_ip = distinct_ip.count()
ips_training = distinct_ip.orderBy(F.rand()).limit(round(tot_num_ip*0.7))

# 70% of tr_raw is going to be used for training.
# 30% of tr_raw is going to be used for validation. 
tr_training = tr_raw.join(ips_training, 'ip')
tr_val = tr_raw.join(ips_training, 'ip', 'left_anti')

In [23]:
# Time-stamp data are converted to Unix times.
unixTimeTrans = SQLTransformer(
    statement="SELECT *, unix_timestamp(click_time) AS click_time_unix \
               FROM __THIS__")

# previous time stamp are created by using window function. 
prevTimeTrans = SQLTransformer(
    statement="SELECT *, LAG(click_time_unix) OVER (PARTITION BY ip ORDER BY click_time_unix ASC) \
                                                    AS prev_click_time_unix \
               FROM __THIS__")

# Time interval between two consecutive clicks from the same ip are computed. 
intervalTimeTrans = SQLTransformer(
    statement="SELECT *, (CASE WHEN prev_click_time_unix IS NULL \
                               THEN 0 \
                               ELSE click_time_unix - prev_click_time_unix END) AS click_interval \
               FROM __THIS__")

# For each ip, 
# clicks = number of total clicks
# click_time_tot_int = time difference between the first and the last clicks for a given IP.
# conversions = number of app downloads (conversions) 
# conversion rate = number of app downloads / number of ad clicks
# and so on. 

featureTrans = SQLTransformer(
    statement = "SELECT ip, COUNT(*) clicks, \
                        MAX(click_time_unix) - MIN(click_time_unix) as click_time_tot_int, \
                        MEAN(click_interval) as click_time_int_mean, \
                        STDDEV(click_interval) as click_time_int_std, \
                        COUNT(DISTINCT app) as app_count, \
                        COUNT(DISTINCT device) as device_count, \
                        COUNT(DISTINCT os) as os_count, \
                        COUNT(DISTINCT channel) as channel_count, \
                        SUM(is_attributed) as conversions, \
                        SUM(is_attributed)/COUNT(*) as conv_rate\
                 FROM __THIS__ \
                 GROUP BY ip")
pipeline_feature_eng = Pipeline(stages=[unixTimeTrans, prevTimeTrans, intervalTimeTrans, featureTrans])
tr_ip = pipeline_feature_eng.fit(tr_training).transform(tr_training).persist()

The content of the dataframe, tr_ip, is shown here.

In [25]:
display(tr_ip)

ip,clicks,click_time_tot_int,click_time_int_mean,click_time_int_std,app_count,device_count,os_count,channel_count,conversions,conv_rate
1829,52,32407,623.2115384615385,2362.8606402736596,17,2,4,31,1,0.019230769230769232
5518,2292,259004,113.00349040139616,500.0820072303256,42,14,49,102,3,0.0013089005235602095
6336,20882,259170,12.411167512690355,34.29575244657039,86,15,79,123,22,0.0010535389330523896
6620,4034,259101,64.22930094199306,243.1952309997336,44,9,50,111,5,0.0012394645513138325
7240,401,85730,213.79052369077306,684.3758696871004,28,3,18,72,1,0.0024937655860349127
7754,5312,259042,48.76543674698795,173.01248031784507,57,11,54,116,8,0.0015060240963855422
7880,5512,258712,46.93613933236575,189.58568485952574,55,11,52,119,8,0.001451378809869376
8086,4496,258992,57.604982206405694,291.9706929265484,41,11,49,110,6,0.0013345195729537367
9376,4124,258591,62.70392822502425,270.0170265722946,54,8,52,117,8,0.0019398642095053346
9427,1513,171407,113.28949107732981,474.9601216920122,38,5,41,95,1,6.609385327164573E-4


In [26]:
display(tr_ip)

ip,clicks,click_time_tot_int,click_time_int_mean,click_time_int_std,app_count,device_count,os_count,channel_count,conversions,conv_rate
1829,52,32407,623.2115384615385,2362.8606402736596,17,2,4,31,1,0.019230769230769232
5518,2292,259004,113.00349040139616,500.0820072303256,42,14,49,102,3,0.0013089005235602095
6336,20882,259170,12.411167512690355,34.29575244657039,86,15,79,123,22,0.0010535389330523896
6620,4034,259101,64.22930094199306,243.1952309997336,44,9,50,111,5,0.0012394645513138325
7240,401,85730,213.79052369077306,684.3758696871004,28,3,18,72,1,0.0024937655860349127
7754,5312,259042,48.76543674698795,173.01248031784507,57,11,54,116,8,0.0015060240963855422
7880,5512,258712,46.93613933236575,189.58568485952574,55,11,52,119,8,0.001451378809869376
8086,4496,258992,57.604982206405694,291.9706929265484,41,11,49,110,6,0.0013345195729537367
9376,4124,258591,62.70392822502425,270.0170265722946,54,8,52,117,8,0.0019398642095053346
9427,1513,171407,113.28949107732981,474.9601216920122,38,5,41,95,1,6.609385327164573E-4


Now, let's move on to the ML training and prediction. First, I define features to use.

In [28]:
feature_cols = tr_ip.columns[1:-2]

In [29]:
# Another feature engineering
# To make the feature vectors and standardize individual feature having its mean value equal to 0 and its standard deviation equal to 1. 

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)

# Train a RandomForest model.
# RF default values: 
# maxDepth=5, 
# maxBins=32, 
# minInstancesPerNode=1, 
# minInfoGain=0.0, 
# maxMemoryInMB=256, 
# cacheNodeIds=False, 
# checkpointInterval=10, 
# impurity='gini', 
# numTrees=20, 
# featureSubsetStrategy='auto', 
# seed=None, 
# subsamplingRate=1.0

rf = RandomForestRegressor(featuresCol="scaledFeatures", labelCol = "conv_rate")
pipeline = Pipeline(stages=[pipeline_feature_eng, assembler, scaler, rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(tr_training)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2031858062450581> in <module> () 
 28 # # Save and load model 
 29 # model.save(sc, "target/tmp/myRandomForestRegressionModel") 
 ---> 30 model . save ( "/mnt/dbdata/fraud/kk_pipeline_model" ) 
 31 # sameModel = RandomForestModel.load(sc, "target/tmp/myRandomForestRegressionModel") 

 /databricks/spark/python/pyspark/ml/pipeline.py in save (self, path) 
 215 def save ( self , path ) : 
 216 """Save this ML instance to the given path, a shortcut of `write().save(path)`.""" 
 --> 217 self . write ( ) . save ( path ) 
 218 
 219 @ classmethod 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 105 if not isinstance ( path , basestring ) : 
 106 raise TypeError ( "path should be a basestring, got type %s" % type ( path ) ) 
 --> 107 self . _jwrite . save ( path ) 
 108 
 109 def overwrite ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1131 answer = self . gateway_client . send_command ( command ) 
 1132 return_value = get_return_value(
 -> 1133 answer, self.gateway_client, self.target_id, self.name)
 1134 
 1135 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.4-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 317 raise Py4JJavaError(
 318 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 319 format(target_id, ".", name), value)
 320 else : 
 321 raise Py4JError(

 Py4JJavaError : An error occurred while calling o459.save.
: java.io.IOException: Path /mnt/dbdata/fraud/kk_pipeline_model already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:109)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:293)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:226)
	at java.lang.Thread.run(Thread.java:748)

After the training, I fed the test data set into the model to predict the conversion rate, and compared the predicted values with the actual values. 
As the plot shows, the predicted values are quite similar to the actual values.

In [31]:
# tr_val.write.parquet("/mnt/dbdata/fraud/kk_test_data")

In [32]:
predictions = model.transform(tr_val)
predictions.select("prediction", "conv_rate", "features").show(5)
predictions.persist()

conv_rate,prediction
0.0021929824561403508,0.0034702183973222772
0.0018050541516245488,0.0034702183973222772
4.2589437819420784E-4,0.0034702183973222772
7.855459544383347E-4,0.0034702183973222772
4.62962962962963E-4,0.0034702183973222772
4.1736227045075126E-4,0.005744398661861322
0.004484304932735426,0.011629052881632273
9.937065253395165E-4,0.0034702183973222772
0.0012463647694225177,0.0034702183973222772
0.0013406323315830633,0.0034702183973222772


In [33]:
display(predictions.select('conv_rate', 'prediction'))

conv_rate,prediction
0.0021929824561403508,0.0034702183973222772
0.0018050541516245488,0.0034702183973222772
4.2589437819420784E-4,0.0034702183973222772
7.855459544383347E-4,0.0034702183973222772
4.62962962962963E-4,0.0034702183973222772
4.1736227045075126E-4,0.005744398661861322
0.004484304932735426,0.011629052881632273
9.937065253395165E-4,0.0034702183973222772
0.0012463647694225177,0.0034702183973222772
0.0013406323315830633,0.0034702183973222772


R square value was 0.91.

In [35]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="conv_rate", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="conv_rate", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("R Square (R^2) on test data = %g" % r2)

rfModel = model.stages[-1]
print(rfModel)  # summary only

Root Mean Squared Error (RMSE) on test data = 0.104385
R Square (R^2) on test data = 0.911764
RandomForestRegressionModel (uid=RandomForestRegressor_4515a30824b003edab1b) with 20 trees

The entire ML pipeline is 

Pipeline(stages=[unixTimeTrans, prevTimeTrans, intervalTimeTrans, featureTrans, assembler, scaler, rf]) 

with the raw data as an input.

In [37]:
# Save and load model
model.save("/mnt/dbdata/fraud/kk_pipeline_model")

from pyspark.ml import PipelineModel
loaded_model = PipelineModel.load("/mnt/dbdata/fraud/kk_pipeline_model")

In [38]:
loaded_model?

Type: PipelineModel
 String form: PipelineModel_49839dd208698fa77cec
 File: /databricks/spark/python/pyspark/ml/pipeline.py
 Docstring: 
Represents a compiled pipeline with transformers and fitted models.

.. versionadded:: 1.3.0

In [39]:
yhat = loaded_model.transform(tr_val)

In [40]:
x.show()

+-----+------+------------------+-------------------+------------------+---------+------------+--------+-------------+-----------+--------------------+--------------------+--------------------+--------------------+
 ip|clicks|click_time_tot_int|click_time_int_mean|click_time_int_std|app_count|device_count|os_count|channel_count|conversions| conv_rate| features| scaledFeatures| prediction|
+-----+------+------------------+-------------------+------------------+---------+------------+--------+-------------+-----------+--------------------+--------------------+--------------------+--------------------+
 5300| 1108| 171974| 155.21119133574007| 673.9492880042757| 35| 6| 38| 92| 2|0.001805054151624...|[1108.0,171974.0,...|[0.09958971420607...|0.003227432407714...|
 7554| 4069| 258695| 63.57704595723765| 273.9711311472205| 55| 9| 51| 108| 4|9.830425165888424E-4|[4069.0,258695.0,...|[0.75086457544142...|0.003227432407714...|
12940| 990| 258034| 260.64040404040406|2802.4091841797776| 44| 3| 37| 93| 4| 0.00404040404040404|[990.0,258034.0,2...|[0.07363549818926...|0.003227432407714...|
13285| 175| 84213| 481.21714285714285|1515.8460551529713| 24| 1| 13| 57| 1|0.005714285714285714|[175.0,84213.0,48...|[-0.1056245531132...|0.003610450043488628|
18654| 2506| 171989| 68.63088587390263|259.00756820409595| 43| 6| 43| 106| 2|7.980845969672786E-4|[2506.0,171989.0,...|[0.40708118871030...|0.003227432407714...|
22346| 2396| 47091| 19.65400667779633| 47.4647003063493| 42| 6| 58| 116| 1|4.173622704507512...|[2396.0,47091.0,1...|[0.38288658055903...|0.003227432407714...|
29744| 1586| 171368| 108.05044136191677| 571.7425298877587| 35| 4| 34| 96| 2|0.001261034047919...|[1586.0,171368.0,...|[0.20472628417246...|0.003227432407714...|
32460| 1621| 259143| 159.8661320172733|2179.5856594506936| 34| 6| 52| 90| 2|0.001233806292412...|[1621.0,259143.0,...|[0.21242456858423...|0.003227432407714...|
33569| 2407| 258720| 107.48649771499792| 305.9198489713765| 42| 7| 58| 106| 3|0.001246364769422...|[2407.0,258720.0,...|[0.38530604137416...|0.003227432407714...|
37111| 8951| 259104| 28.946933303541503|119.33272193490438| 59| 9| 54| 122| 12|0.001340632331583...|[8951.0,259104.0,...|[1.82466527539108...|0.003227432407714...|
40574| 293| 83699| 285.66211604095565| 954.5311076352892| 24| 1| 21| 62| 1|0.003412969283276...|[293.0,83699.0,28...|[-0.0796703370964...|0.003227432407714...|
43935| 337| 85978| 255.12759643916914| 852.5937553412437| 31| 4| 31| 61| 1|0.002967359050445104|[337.0,85978.0,25...|[-0.0699924938359...|0.003227432407714...|
46943| 1765| 172471| 97.71728045325779|322.01765941046085| 34| 7| 48| 96| 3|0.001699716713881...|[1765.0,172471.0,...|[0.24409751016406...|0.003227432407714...|
51393| 15| 46643| 3109.5333333333333| 5807.801549475949| 7| 2| 4| 8| 1| 0.06666666666666667|[15.0,46643.0,310...|[-0.1408167104241...| 0.07660289753243607|
56054| 1470| 171026| 116.34421768707483| 518.3785750360435| 39| 6| 38| 100| 4|0.002721088435374...|[1470.0,171026.0,...|[0.17921197012204...|0.003227432407714...|
63645| 568| 256585| 451.73415492957747|3756.9395596852078| 29| 9| 38| 78| 3| 0.00528169014084507|[568.0,256585.0,4...|[-0.0191838167183...|0.003227432407714...|
67782| 2664| 258674| 97.09984984984985|1720.3560861786573| 38| 6| 46| 96| 0| 0.0|[2664.0,258674.0,...|[0.44183344405484...|0.003227432407714...|
70097| 1977| 258447| 130.7268588770865| 436.5384473839964| 41| 9| 48| 95| 3|0.001517450682852...|[1977.0,258447.0,...|[0.29072711860104...|0.003227432407714...|
73933| 38674| 259180| 6.701660029994311|21.644339506307215| 91| 23| 71| 143| 113|0.002921859647308...|[38674.0,259180.0...|[8.36226834884538...|0.003227432407714...|
76885| 40248| 259195| 6.439947326575234|13.516882446824885| 120| 64| 104| 141| 37|9.193003379049891E-4|[40248.0,259195.0...|[8.70847119639163...|0.003227432407714...|
+-----+------+------------------+-------------------+------------------+---------+------------+--------+-------------+-----------+--------------------+--------------------+----

In [41]:
loaded_model.stages

Out[ 53 ]: 
[PipelineModel_41f98dd199e372967fa5,
 VectorAssembler_49f8b22bec3c0d4f09b6,
 StandardScaler_408992bf1a5eb502d551,
 RandomForestRegressionModel (uid=RandomForestRegressor_4272b57e0c445ef34c52) with 20 trees]

In [42]:
loaded_model.stages[0].stages

Out[ 54 ]: 
[SQLTransformer_485385b5c1bf1721790c,
 SQLTransformer_4e6caf76b0b9512cdc79,
 SQLTransformer_43b9852f2beb6cc0dbdd,
 SQLTransformer_42a5a3fa7cdab71db6ac]

In [43]:
loaded_model.stages[0].transform(tr_val).show()

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-2662351500073115> in <module> () 
 ----> 1 Pipeline ( loaded_model . stages [ 0 : 1 ] ) . transform ( tr_val ) . show ( ) 

 /databricks/spark/python/pyspark/__init__.py in wrapper (self, *args, **kwargs) 
 100 def wrapper ( self , * args , ** kwargs ) : 
 101 if len ( args ) > 0 : 
 --> 102 raise TypeError ( "Method %s forces keyword arguments." % func . __name__ ) 
 103 self . _input_kwargs = kwargs 
 104 return func ( self , ** kwargs ) 

 TypeError : Method __init__ forces keyword arguments.

In [44]:
pipeline.save("/mnt/dbdata/fraud/kk_pipeline_unfit")

In [45]:
loaded_pipeline = Pipeline.load("/mnt/dbdata/fraud/kk_pipeline_unfit")

In [46]:
loaded_pipeline